# PyTorch 多文件项目演示

这个 Notebook 演示如何在 Google Colab 中使用多文件 PyTorch 项目


## 1. 环境设置


In [ ]:
# ========================================
# 🚀 从 GitHub 克隆项目（推荐方式）
# ========================================

import os

print("=" * 60)
print("📦 UCAS PyTorch 项目启动")
print("=" * 60)

# 检查项目是否已存在
if os.path.exists('/content/UCAS'):
    print("\n✅ 项目已存在，拉取最新代码...")
    %cd /content/UCAS
    !git pull origin main
else:
    print("\n📥 首次克隆项目...")
    !git clone https://github.com/AresNan7/UCAS.git
    %cd UCAS

print(f"\n📂 当前目录: {os.getcwd()}")

# 查看项目结构
print("\n📁 项目文件:")
!ls -la

print("\n📁 Models 目录:")
!ls models/

print("\n📁 Utils 目录:")
!ls utils/

# 安装依赖
print("\n📦 安装依赖包...")
!pip install -q torch torchvision torchaudio tqdm numpy matplotlib

print("\n🎉 环境准备完成！")


In [ ]:
# 检查环境和测试模块导入
import torch
import sys

print("=" * 60)
print("🔍 环境检查")
print("=" * 60)

print(f"\nPython 版本: {sys.version}")
print(f"PyTorch 版本: {torch.__version__}")
print(f"CUDA 可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA 设备: {torch.cuda.get_device_name(0)}")
    print(f"CUDA 版本: {torch.version.cuda}")
    print(f"显存: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# 测试模块导入
print("\n" + "=" * 60)
print("📚 测试模块导入")
print("=" * 60)

try:
    from models import SimpleNN, SimpleCNN, AdvancedCNN, ResNet18, ResNet34, SimpleLSTM, BidirectionalLSTM
    print("✅ 模型模块导入成功")
except Exception as e:
    print(f"❌ 模型导入失败: {e}")

try:
    from utils import train_epoch, validate, test, save_checkpoint, load_checkpoint
    from utils import get_mnist_loaders, get_cifar10_loaders, create_synthetic_dataset
    from utils import EarlyStopping, MetricTracker
    print("✅ 工具模块导入成功")
except Exception as e:
    print(f"❌ 工具导入失败: {e}")

print("\n" + "=" * 60)
print("🎉 所有模块准备就绪，可以开始实验！")
print("=" * 60)


## 2. 快速测试模型


In [ ]:
# 测试所有模型的前向传播
import torch
from models import SimpleNN, SimpleCNN, ResNet18, SimpleLSTM

print("🧪 测试所有模型...\n")

# 1. SimpleNN (MLP)
model = SimpleNN(input_size=784, hidden_sizes=[256, 128], num_classes=10)
x = torch.randn(4, 1, 28, 28)
output = model(x)
print(f"✅ SimpleNN - 输入: {x.shape}, 输出: {output.shape}, 参数: {model.get_num_params():,}")

# 2. SimpleCNN
model = SimpleCNN(num_classes=10, in_channels=1)
output = model(x)
print(f"✅ SimpleCNN - 输入: {x.shape}, 输出: {output.shape}, 参数: {sum(p.numel() for p in model.parameters()):,}")

# 3. ResNet18
model = ResNet18(num_classes=10, in_channels=3)
x = torch.randn(4, 3, 32, 32)
output = model(x)
print(f"✅ ResNet18 - 输入: {x.shape}, 输出: {output.shape}, 参数: {sum(p.numel() for p in model.parameters()):,}")

# 4. SimpleLSTM
model = SimpleLSTM(vocab_size=1000, num_classes=5)
x = torch.randint(0, 1000, (4, 50))
output, _ = model(x)
print(f"✅ SimpleLSTM - 输入: {x.shape}, 输出: {output.shape}, 参数: {sum(p.numel() for p in model.parameters()):,}")

print("\n🎉 所有模型测试通过！")


## 3. 完整训练示例


In [ ]:
# 使用合成数据进行快速训练演示
import torch
import torch.nn as nn
import torch.optim as optim
from models import SimpleNN
from utils import create_synthetic_dataset, train_epoch, validate, MetricTracker

print("=" * 60)
print("🏃 快速训练演示（使用合成数据）")
print("=" * 60)

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n🖥️  使用设备: {device}")

# 创建合成数据
print("\n📊 创建数据...")
train_loader, test_loader = create_synthetic_dataset(
    num_samples=1000, 
    input_dim=20, 
    num_classes=5
)
print(f"✅ 训练批次: {len(train_loader)}, 测试批次: {len(test_loader)}")

# 创建模型
print("\n🧠 创建模型...")
model = SimpleNN(input_size=20, hidden_sizes=[64, 32], num_classes=5).to(device)
print(f"✅ 模型参数: {model.get_num_params():,}")

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练
print("\n🏃 开始训练...")
num_epochs = 10
tracker = MetricTracker()

for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(
        model, train_loader, criterion, optimizer, device, verbose=False
    )
    val_loss, val_acc = validate(
        model, test_loader, criterion, device, verbose=False
    )
    
    tracker.update(train_loss, train_acc, val_loss, val_acc)
    
    print(f"Epoch {epoch+1:2d}/{num_epochs} - "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:5.2f}% | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:5.2f}%")

# 打印总结
print("\n" + "=" * 60)
tracker.print_summary()
print("=" * 60)


## 4. 可视化训练结果


In [ ]:
# 绘制训练过程的损失和准确率曲线
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

# 损失曲线
plt.subplot(1, 2, 1)
plt.plot(tracker.train_losses, 'b-', label='训练损失', linewidth=2)
plt.plot(tracker.val_losses, 'r-', label='验证损失', linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend(fontsize=10)
plt.title('损失曲线', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# 准确率曲线
plt.subplot(1, 2, 2)
plt.plot(tracker.train_accs, 'b-', label='训练准确率', linewidth=2)
plt.plot(tracker.val_accs, 'r-', label='验证准确率', linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Accuracy (%)', fontsize=12)
plt.legend(fontsize=10)
plt.title('准确率曲线', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 最佳验证准确率: {max(tracker.val_accs):.2f}% (Epoch {tracker.val_accs.index(max(tracker.val_accs)) + 1})")


## 5. 更多示例和说明


In [ ]:
# 💡 更多使用示例

print("=" * 60)
print("📚 其他可用功能")
print("=" * 60)

# 1. 运行完整测试套件
print("\n1️⃣ 运行自动化测试:")
print("   !python test_modules.py")

# 2. 使用训练脚本
print("\n2️⃣ 使用命令行训练脚本:")
print("   !python train.py --model simple_nn")
print("   !python train.py --model cnn")
print("   !python train.py --model resnet")
print("   !python train.py --model lstm")

# 3. 使用真实数据集
print("\n3️⃣ 使用真实数据集:")
print("   from utils import get_mnist_loaders, get_cifar10_loaders")
print("   train_loader, test_loader = get_mnist_loaders(batch_size=128)")

# 4. 保存和加载模型
print("\n4️⃣ 保存和加载模型:")
print("   from utils import save_checkpoint, load_checkpoint")
print("   save_checkpoint(model, optimizer, epoch, loss, acc, 'model.pth')")

# 5. 更新代码
print("\n5️⃣ 从 GitHub 拉取最新代码:")
print("   %cd /content/UCAS")
print("   !git pull origin main")

# 6. 查看文档
print("\n6️⃣ 查看项目文档:")
print("   !cat README_PYTORCH.md")
print("   !cat QUICKSTART.md")

print("\n" + "=" * 60)


## 6. 总结

这个 Notebook 演示了如何在 Colab 中使用多文件 PyTorch 项目：

1. ✅ 导入自定义模块（models 和 utils）
2. ✅ 测试不同类型的模型（MLP、CNN、ResNet、LSTM）
3. ✅ 完整的训练流程
4. ✅ 可视化训练结果
5. ✅ 模型保存和加载

你可以根据需要修改和扩展这些代码！

### 更多示例

参考 `train.py` 查看更多训练示例：
- SimpleNN on MNIST
- CNN on CIFAR-10
- ResNet on CIFAR-10
- LSTM on 序列数据

查看 `README_PYTORCH.md` 了解详细的文档和使用说明。
